In [ ]:
%%capture
!pip install boto3
!pip install Levenshtein
!pip install python-Levenshtein
!pip install azure-cognitiveservices-vision-computervision boto3
!apt-get install -y tesseract-ocr
!pip install pytesseract

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
import json
import random
import time
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import requests
import boto3
from difflib import SequenceMatcher
import logging
from openai import OpenAI
import Levenshtein
import pytesseract
from PIL import Image

In [ ]:
folder = "/content/drive/MyDrive/vision_datasets/SROIE2019"

#folder="/content/test_data/test"
#folder=r"C:\Users\franc\OneDrive\Desktop\vision project\ICDAR FOCUSED SCENE TEXT"

In [ ]:
max_size = 1 * 1024 * 1024

image_folder=os.path.join(folder,"task1&2_test(361p)")
text_folder=os.path.join(folder,"text.task1&2-test（361p)")

images_paths = sorted(
    [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith((".jpg", ".png", ".jpeg")) and os.path.getsize(os.path.join(image_folder, file)) < max_size]
)


In [ ]:
import re

In [ ]:
images_paths

['/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X00016469670.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X00016469671.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005200931.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005230605.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005230616.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005230621.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005230648.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005230657.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005230659.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005268275.jpg',
 '/content/drive/MyDrive/vision_datasets/SROIE2019/task1&2_test(361p)/X51005268408.jpg',
 '/content/drive/MyDr

In [ ]:
# Funzione per estrarre parole da un file di testo


def estrai_parole_da_file(percorso_file):
    parole = []
    with open(percorso_file, 'r') as file:
        for riga in file:
            # Extract the last word or quoted word after the last comma
            match = re.search(r',\s*"?([^",]+)"?\s*$', riga.strip())
            if match:
                transcription = match.group(1)
                # Ignore transcriptions that are ###
                if transcription != "###":
                    parole.append(transcription)
    return parole
#ESEMPIO
# Percorso al file di testo
percorso_file = "/content/drive/MyDrive/vision_datasets/SROIE2019/text.task1&2-test（361p)/X51009447842.txt"

# Estrazione delle parole
parole_estratte = estrai_parole_da_file(percorso_file)
print(parole_estratte)

['PASARAYA BORONG PINTAR', 'SDN BHD', 'BR NO.: (124525-H)', 'MEDAN NIAGA TASIK DAMAI', 'GST NO.:', '14/03/2018 4:06:34PM', 'TAX INVOICE', 'TRN: CR0008955', 'COUNTER 3', 'CASHIER: 1', 'QTY UOM', 'UNIT AMT EXC. AMT INC. GST', 'PRICE', 'TAX', 'TAX CODE', 'GB CR-CORN', '1', 'WALK', '0.90', '0.90', '0.90', 'ZRL', '*TOTAL QTY: 1.00', '0.90', 'TOTAL INCLUDES GST 0%', '0.90', "CUSTOMER'S PAYMENT", 'CASH', 'CHANGE', '5.00', '4.10', 'GST SUMMARY', 'AMOUNT (RM)', 'TAX (RM)', 'ZRL = 0 %', '0.90', '0.00', '*GOODS SOLD ARE NOT RETURNABLE', 'THANK YOU FOR SHOPPING AT PASARAYA', 'BORONG PINTAR SDN BHD']


# LLM Initializitation

In [ ]:


import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [ ]:
%pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [ ]:
prompt_template='''You are an advanced language model specialized in text correction. You will receive text extracted from images using OCR (Optical Character Recognition) models. Your task is to identify and correct any misspelled words or inaccuracies while preserving the context and intended meaning of the text.

### Instructions:
1. Correct spelling errors: Replace any misspelled words with the correct ones.
2. Maintain context: Ensure that the corrected text aligns with the overall meaning and structure of the original input.
3. Handle OCR-specific errors:
   - Fix common OCR mistakes such as incorrect substitutions of similar-looking characters (e.g., "rn" misread as "m").
   - Handle mixed-case errors, such as "tHiS iS" to "This is."
4. Do not alter proper nouns, numbers, or special characters unless they are obviously incorrect.
5. Do not add or remove punctuations

Output the corrected text clearly and concisely.

### Example Inputs and Outputs:

**Input:**
Th1s 1s an exarnple of OCR t3xt.

**Output:**
This is an example of OCR text.

**Input:**
Th@ rn0del w1ll c0rrect err0rs.

**Output:**
The model will correct errors.

Focus on accuracy and consistency. Your goal is to produce clean and coherent text that closely resembles the intended content. Only output the corrected text and noting else'''

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            prompt_template,
        ),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain_core.output_parsers.string import StrOutputParser

Esempio:

In [ ]:
model=prompt|llm|StrOutputParser()

model.invoke({'input':"Hrllo,my name is john"})

'Hello, my name is John'

In [ ]:

# Functions for Metrics
def calculate_levenshtein_accuracy(extracted_text, ground_truth):
    extracted_text = extracted_text.strip().lower()
    ground_truth = ground_truth.strip().lower()
    distance = Levenshtein.distance(extracted_text, ground_truth)
    max_length = max(len(extracted_text), len(ground_truth))
    return (1 - distance / max_length) * 100 if max_length > 0 else 0

def calculate_word_accuracy(expected_words, ocr_words):
    expected_set = set(word.lower() for word in expected_words)
    ocr_set = set(word.lower() for word in ocr_words)
    matching_words = expected_set & ocr_set
    return (len(matching_words) / len(expected_set)) * 100 if expected_set else 0

def calculate_character_error_rate(extracted_text, ground_truth):
    errors = Levenshtein.distance(extracted_text, ground_truth)
    total_chars = len(ground_truth)
    return (errors / total_chars) * 100 if total_chars > 0 else 0

# Evaluation Pipeline
def evaluate_ocr(ocr_dict,type):
    """
    Evaluate OCR output against ground truth for each entry in a dictionary.

    Parameters:
        ocr_dict (dict): Dictionary with keys as image names and values as OCR outputs.
        type: 'scene' o 'document'

    Returns:
        dict: A dictionary with image names as keys and evaluation metrics as values.
    """
    evaluation_results = {}
    #image_folder_path = os.path.join(folder, "Images")
    #text_folder_path = os.path.join(folder, "Text")

    for image_name, ocr_output in ocr_dict.items():

        #ground_truth_file = f"{folder}/{os.path.splitext(image_name)[0]}.json"
        if type=="scene":
            ground_truth_file=os.path.join(folder,'gt_'+ image_name.rsplit('.', 1)[0] + '.txt')
        if type=="document":
            ground_truth_file=os.path.join(text_folder, image_name.rsplit('.', 1)[0] + '.txt')

      #output_json_path = os.path.join(folder, image_name.rsplit('.', 1)[0] + '_ocr.json')



        # Assuming ground truth JSON has a key "text" containing the expected output
        #ground_truth = " ".join(item["text"] for item in ground_truth_data if "text" in item)

        ground_truth_words = estrai_parole_da_file(ground_truth_file)
        ground_truth_text=" ".join(item for item in ground_truth_words)
        lev_acc_baseline = calculate_levenshtein_accuracy(ocr_output, ground_truth_text)
        word_acc_baseline = calculate_word_accuracy(ground_truth_words, ocr_output.split())
        cer_baseline = calculate_character_error_rate(ocr_output, ground_truth_text)

        #LLM
        llm_output=model.invoke({'input':ocr_output})
        time.sleep(5)#Con il piano gratuito, possiamo fare solo 15 run al minuto
        lev_acc_llm = calculate_levenshtein_accuracy(llm_output, ground_truth_text)
        word_acc_llm = calculate_word_accuracy(ground_truth_words, llm_output.split())
        cer_llm = calculate_character_error_rate(llm_output, ground_truth_text)



        # Store the results in the evaluation dictionary
        evaluation_results[os.path.splitext(image_name)[0]] = {
            "Levenshtein Accuracy": lev_acc_baseline,
            "Word Accuracy": word_acc_baseline,
            "Character Error Rate": cer_baseline,
            "Levenshtein Accuracy LLM": lev_acc_llm,
            "Word Accuracy LLM": word_acc_llm,
            "Character Error Rate LLM": cer_llm

        }
    return evaluation_results

## AZURE

In [ ]:
subscription_key = AZURE_API_KEY
endpoint = "https://patrikbaldon.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [ ]:
azure_extraction = {}

# Funzione per estrarre testo e aggiungerlo al dizionario
def extract_text_azure(image_path):
    with open(image_path, "rb") as image_file:
        read_response = computervision_client.read_in_stream(image_file, raw=True)
        operation_location = read_response.headers["Operation-Location"]
        operation_id = operation_location.split("/")[-1]

        # Attendi il completamento
        while True:
            result = computervision_client.get_read_result(operation_id)
            if result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        # Estrai il testo
        extracted_text = ""
        if result.status == OperationStatusCodes.succeeded:
            for page in result.analyze_result.read_results:
                for line in page.lines:
                    extracted_text += line.text + " "

        # Salva nel dizionario
        azure_extraction[os.path.basename(image_path)] = extracted_text.strip()

In [ ]:
# Processa ogni immagine
azure_time_record=[]
for image_path in images_paths[0:150]:
    start_time = time.time()
    extract_text_azure(image_path)
    end_time = time.time()
    elapsed_time = end_time - start_time
    azure_time_record.append(elapsed_time)

In [ ]:
Azure_Evaluation = evaluate_ocr(azure_extraction,'document')

In [ ]:
keys = list(Azure_Evaluation.keys())
values = list(Azure_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")


First record:
Key: X00016469670
Value: {'Levenshtein Accuracy': 89.30163447251115, 'Word Accuracy': 32.5, 'Character Error Rate': 40.88669950738916, 'Levenshtein Accuracy LLM': 89.45022288261517, 'Word Accuracy LLM': 32.5, 'Character Error Rate LLM': 40.88669950738916}


# AWS

In [ ]:
# Configura manualmente le credenziali e la regione
session = boto3.Session(
    aws_access_key_id=AWS_API_KEY,
    aws_secret_access_key=AWS_SECRET,
    region_name='us-east-1'
)
bucket_name = "patriksbucket"
# Esempio: utilizza il client Textract
# Inizializza i client con la sessione
s3 = session.client('s3')
textract = session.client('textract')
response = s3.list_objects_v2(Bucket=bucket_name, Prefix="SROIE2019/task1&2_test(361p)/")

In [ ]:
file_names = [os.path.splitext(os.path.basename(path))[0] for path in images_paths]
print(file_names)

['X00016469670', 'X00016469671', 'X51005200931', 'X51005230605', 'X51005230616', 'X51005230621', 'X51005230648', 'X51005230657', 'X51005230659', 'X51005268275', 'X51005268408', 'X51005288570', 'X51005301666', 'X51005337867', 'X51005337877', 'X51005361906', 'X51005361912', 'X51005361923', 'X51005365187', 'X51005433518', 'X51005433543', 'X51005433548', 'X51005433556', 'X51005442322', 'X51005442334', 'X51005442343', 'X51005442366', 'X51005442375', 'X51005442382', 'X51005442388', 'X51005444040', 'X51005444041', 'X51005444044', 'X51005444046', 'X51005447841', 'X51005447842', 'X51005447844', 'X51005447851', 'X51005447859', 'X51005568855', 'X51005568866', 'X51005568885', 'X51005568887', 'X51005568889', 'X51005568890', 'X51005568892', 'X51005568894', 'X51005568895', 'X51005577191', 'X51005582745', 'X51005587261', 'X51005605287', 'X51005605295', 'X51005605332', 'X51005621482', 'X51005663274', 'X51005663300', 'X51005663307', 'X51005663309', 'X51005663310', 'X51005663323', 'X51005675099', 'X51005

In [ ]:
def list_images_in_folder(bucket):
    session = boto3.Session(
        aws_access_key_id=AWS_API_KEY,
        aws_secret_access_key=AWS_SECRET,
        region_name='us-east-1'
    )
    s3 = session.client('s3')
    response = s3.list_objects_v2(Bucket=bucket, Prefix="SROIE2019/task1&2_test(361p)/")

    # Dimensione massima in bytes (1 MB)
    max_size = 1 * 1024 * 1024  # 1 MB

    # Ottieni e ordina le immagini per nome
    sorted_objects = sorted(
        [obj for obj in response.get('Contents', []) if obj['Key'].endswith(('.jpg', '.png', '.jpeg'))],
        key=lambda x: x['Key']
    )

    # Filtra immagini per dimensione
    return [
        (obj['Key'], obj['Key'].split('/')[-1].split('.')[0])
        for obj in sorted_objects
        if (obj['Size'] < max_size and obj['Key'].split('/')[-1].split('.')[0] in file_names)
    ]


In [ ]:
def extract_text_textract(bucket, image_name):
    session = boto3.Session(
        aws_access_key_id=AWS_API_KEY,
        aws_secret_access_key=AWS_SECRET,
        region_name='us-east-1'
    )
    textract = session.client('textract')
    response = textract.detect_document_text(
        Document={'S3Object': {'Bucket': bucket, 'Name': image_name}}
    )
    extracted_text = []
    for block in response['Blocks']:
        if block['BlockType'] == 'LINE':
            extracted_text.append(block['Text'])
    return " ".join(extracted_text)

In [ ]:
# Configurazione AWS
region_name = "us-east-1"
bucket_name = "patriksbucket"

aws_extraction = {}


files = list_images_in_folder(bucket_name)

aws_time_record=[]
for path, image_name in files[0:150]:
  # Estrai testo dall'immagine
  start_time = time.time()
  aws_extraction[image_name] = extract_text_textract(bucket_name, path)
  end_time = time.time()
  elapsed_time = end_time - start_time
  aws_time_record.append(elapsed_time)

In [ ]:
aws_extraction

{'X00016469670': 'tan chay yee *** COPY *** OJC MARKETING SDN BHD ROC NO: 538358-H NO 2 & 4, JALAN BAYU 4, BANDAR SERI ALAM, 81750 MASAI, JOHOR Tel:07-388 2218 Fax:07-388 8218 Email: ng@ojcgroup.com TAX INVOICE Invoice No : PEGIV-1030765 Date : 15/01/2019 11:05:16 AM Cashier : NG CHUAN MIN Sales Person : FATIN Bill To : THE PEAK QUARRY WORKS Address :. Description Qty Price Amount 000000111 1 193.00 193.00 SR KINGS SAFETY SHOES KWD 805 Qty: 1 Total Exclude GST: 193.00 Total GST @6%: 0.00 Total Inclusive GST: 193.00 Round Amt: 0.00 TOTAL: 193.00 VISA CARD 193.00 xxxxxxxxxxxx4318 Approval Code:000 193.00 Goods Sold Are Not Returnable & Refundable **** Thank You. Please Come Again. ****',
 'X00016469671': 'tan chay yee OJC MARKETING SDN BHD ROC NO: 538358-H NO 2 & 4, JALAN BAYU 4, BANDAR SERI ALAM, 81750 MASAI, JOHOR Tel:07-388 2218 Fax:07-388 8218 Email: ng@ojcgroup.com Cash Bill Invoice No : PEGIV-1030531 Date 02/01/2019 2:47:14 PM Cashier : RHYS TAN Sales Person : FATIN Description Qty

In [ ]:
#folder = "/content/drive/MyDrive/vision_datasets/Handwriting"

In [ ]:
Aws_Evaluation = evaluate_ocr(aws_extraction,'document')

In [ ]:
keys = list(Aws_Evaluation.keys())
values = list(Aws_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: X00016469670
Value: {'Levenshtein Accuracy': 90.20771513353117, 'Word Accuracy': 37.5, 'Character Error Rate': 42.03612479474548, 'Levenshtein Accuracy LLM': 90.07407407407408, 'Word Accuracy LLM': 35.0, 'Character Error Rate LLM': 42.20032840722496}


# OCR_SPACE

In [ ]:
#folder="/content/test_data/test"

In [ ]:
API_KEY = OCR_SPACE_API_KEY
ocrspace_extraction = {}

In [ ]:

def ocr_space_file(image_path, output_path=None, overlay=False, api_key=None, language='eng'):
    """
    OCR.space API request with a local file, optionally saving results as JSON.

    Parameters:
        image_path (str): Path to the file.
        output_path (str, optional): Path to save the resulting JSON file.
        overlay (bool, optional): Include OCR overlay in the response.
        api_key (str): Your OCR.space API key.
        language (str): Language code for OCR.

    Returns:
        dict: A dictionary containing image names as keys and extracted text as values.
    """

    payload = {
        'isOverlayRequired': overlay,
        'apikey': api_key,
        'language': language,
        'OCREngine': 2
    }


    # Open the image file in binary mode
    with open(image_path, 'rb') as f:
      r = requests.post(
      'https://api.ocr.space/parse/image',
      files={os.path.basename(image_path): f},
      data=payload,
      )
      result = r.json()

      # Skip if there are errors in the response
      #if result.get("IsErroredOnProcessing"):
      #  return extracted_text

      ## Extract the text and remove newlines
      extracted_text = result["ParsedResults"][0]["ParsedText"].replace('\n', ' ')
    return extracted_text




In [ ]:
import os
import requests

def ocr_space_file(image_path, output_path=None, overlay=False, api_key=None, language='eng'):
    """
    OCR.space API request with a local file, optionally saving results as JSON.

    Parameters:
        image_path (str): Path to the file.
        output_path (str, optional): Path to save the resulting JSON file.
        overlay (bool, optional): Include OCR overlay in the response.
        api_key (str): Your OCR.space API key.
        language (str): Language code for OCR.

    Returns:
        str: Extracted text from the image or None if an error occurred.
    """
    payload = {
        'isOverlayRequired': overlay,
        'apikey': api_key,
        'language': language,
        'OCREngine': 2
    }

    try:
        # Open the image file in binary mode
        with open(image_path, 'rb') as f:
            r = requests.post(
                'https://api.ocr.space/parse/image',
                files={os.path.basename(image_path): f},
                data=payload,
            )
        result = r.json()

        # Check if the response contains errors
        #if result.get("IsErroredOnProcessing", False):
        #    print(f"Error processing {image_path}: {result.get('ErrorMessage', 'Unknown error')}")
        #    return None

        # Extract the text
        return result["ParsedResults"][0]["ParsedText"].replace('\n', ' ')

    except Exception as e:
        print(f"Exception occurred for {image_path}: {e}")
        return None



In [ ]:
# Process multiple images
ocrspace_extraction = {}
ocr_space_time_record=[]
for image_path in images_paths[0:150]:
    start_time = time.time()
    text = ocr_space_file(image_path, api_key=OCR_SPACE_API_KEY)
    end_time = time.time()
    elapsed_time = end_time - start_time
    ocr_space_time_record.append(elapsed_time)
    time.sleep(2)
    if text:  # Only add if text extraction succeeded
        ocrspace_extraction[os.path.basename(image_path)] = text

In [ ]:
ocrspace_extraction

{'X00016469670.jpg': 'tan chay yee *** COPY *** OJC MARKETING SDN BHD ROC NO: 538358-H NO 2 & 4, JALAN BAYU 4, BANDAR SERI ALAM, 81750 MASAI, JOHOR Tel:07-388 2218 Fax:07-388 8218 Email: ng@ojcgroup.com TAX INVOICE Invoice No : PEGIV-1030765 Date : 15/01/2019 11:05:16 AM Cashier : NG CHUAN MIN Sales Persor : FATIN Bill To : THE PEAK QUARRY WORKS Address Description. 000000111 Qty. 1 Price 193.00 KINGS SAFETY SHOES KWD 805 Qty: 1 Total Exclude GST: Total GST @6%: Total Inclusive GST: Round Amt: TOTAL: VISA CARD xx000000000x4318 Approval Code:000 Amount 193.00 SR 193.00 0.00 193.00 0.00 193.00 193.00 93.00 Goods Sold Are Not Returable & Refundable ****Thank You. Please Come Again.****',
 'X00016469671.jpg': 'tan chay yee OJC MARKETING SDN BHD ROC NO: 538358-H NO 2 & 4, JALAN BAYU 4, BANDAR SERI ALAM, 81750 MASAI, JOHOR Tel:07-388 2218 Fax:07-388 8218 Email: ng@ojcgroup.com Cash Bill Invoice No : PEGIV-1030531 Date 02/01/2019 2:47:14 PM Cashier : RHYS TAN Sales Persor : FATIN Description 

In [ ]:
OcrSpace_Evaluation = evaluate_ocr(ocrspace_extraction , 'document')

In [ ]:
keys = list(OcrSpace_Evaluation.keys())
values = list(OcrSpace_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: X00016469670
Value: {'Levenshtein Accuracy': 70.254110612855, 'Word Accuracy': 27.500000000000004, 'Character Error Rate': 56.97865353037766, 'Levenshtein Accuracy LLM': 70.9145427286357, 'Word Accuracy LLM': 27.500000000000004, 'Character Error Rate LLM': 56.97865353037766}


esempio output ocr space

In [ ]:
ocrspace_extraction

{'img_109.jpg': 'SPIDER MAN',
 'img_11.jpg': "iM. Summer's Here!",
 'img_110.jpg': '123 FANSTORMERS GRNGD2',
 'img_111.jpg': 'NATURAL HISTORY MUSEUM',
 'img_112.jpg': 'CLOSING DOWN SALE',
 'img_113.jpg': 'one hour the eyecare clinic contact lens centre',
 'img_114.jpg': 'act one',
 'img_115.jpg': 'FUN FASHION LADIES LINGERIE SHOP 01206 368166 FUNFASHION@HOTMAIL.COM',
 'img_116.jpg': 'P',
 'img_117.jpg': 'Counselling Helpline Lifelines Family Support'}

# TESSERACT

In [ ]:
tesseract_extraction = {}

In [ ]:
# Function to extract text using Tesseract OCR
def extract_text_tesseract(image_path):
  text = pytesseract.image_to_string(Image.open(image_path))
  return text.strip()

In [ ]:
tesseract_time_record=[]
for image_path in images_paths[0:150]:
  start_time = time.time()
  tesseract_extraction[os.path.basename(image_path)] = extract_text_tesseract(image_path)
  end_time = time.time()
  elapsed_time = end_time - start_time
  tesseract_time_record.append(elapsed_time)

In [ ]:
Tesseract_Evaluation = evaluate_ocr(tesseract_extraction ,'document')

In [ ]:
keys = list(Tesseract_Evaluation.keys())
values = list(Tesseract_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: X00016469670
Value: {'Levenshtein Accuracy': 60.591133004926114, 'Word Accuracy': 17.5, 'Character Error Rate': 65.1888341543514, 'Levenshtein Accuracy LLM': 60.91954022988506, 'Word Accuracy LLM': 12.5, 'Character Error Rate LLM': 65.1888341543514}


# GOOGLE CLOUD VISION API

In [ ]:
!pip install google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.6/514.6 kB 11.7 MB/s eta 0:00:00


In [ ]:
cloudvisionapikey = CLOUD_VISION_API
cloudvision_extraction = {}

In [ ]:
from google.cloud import vision
from google.auth.transport.requests import Request
from google.auth.credentials import AnonymousCredentials
import requests
import base64 # Import base64

def detect_text(image_path):
    """
    Detect text in an image using the Vision API with an API key.

    Args:
        api_key (str): Your Google Cloud Vision API key.
        image_path (str): Path to the image file.

    Returns:
        str: Detected text.
    """
    # Read the image file
    with open(image_path, "rb") as image_file:
        image_content = image_file.read()

    # Encode image content to base64
    image_content_base64 = base64.b64encode(image_content).decode('utf-8')

    # Create the request payload
    url = f"https://vision.googleapis.com/v1/images:annotate?key={cloudvisionapikey}"
    headers = {"Content-Type": "application/json"}
    payload = {
        "requests": [
            {
                "image": {"content": image_content_base64}, # Use base64 encoded content
                "features": [{"type": "TEXT_DETECTION"}]
            }
        ]
    }

    # Send the request
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()

    # Parse the response
    result = response.json()
    if "responses" in result and "textAnnotations" in result["responses"][0]:
        detected_text = result["responses"][0]["textAnnotations"][0]["description"]
        return detected_text
    else:
        return "No text detected."

In [ ]:
google_vision_time_record=[]
for image_path in images_paths[0:150]:
  start_time = time.time()
  cloudvision_extraction[os.path.basename(image_path)] = detect_text(image_path)
  end_time = time.time()
  elapsed_time = end_time - start_time
  google_vision_time_record.append(elapsed_time)

In [ ]:
OcrCloudVision_Evaluation = evaluate_ocr(cloudvision_extraction,'document')

In [ ]:
keys = list(OcrCloudVision_Evaluation.keys())
values = list(OcrCloudVision_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: X00016469670
Value: {'Levenshtein Accuracy': 67.96992481203009, 'Word Accuracy': 32.5, 'Character Error Rate': 60.26272577996716, 'Levenshtein Accuracy LLM': 79.18552036199095, 'Word Accuracy LLM': 32.5, 'Character Error Rate LLM': 51.55993431855501}


# COMPARE THE EVALUATIONS

In [ ]:
import pandas as pd

In [ ]:
def calculate_score(metrics, alpha, beta, gamma):

    lev_accuracy = metrics.get("Levenshtein Accuracy", 0)
    word_accuracy = metrics.get("Word Accuracy", 0)
    cer = metrics.get("Character Error Rate", 0)

    total_score = alpha * lev_accuracy + beta * word_accuracy - gamma * cer
    return total_score

def calculate_score_llm(metrics, alpha, beta, gamma):

    lev_accuracy = metrics.get("Levenshtein Accuracy LLM", 0)
    word_accuracy = metrics.get("Word Accuracy LLM", 0)
    cer = metrics.get("Character Error Rate LLM", 0)

    total_score = alpha * lev_accuracy + beta * word_accuracy - gamma * cer
    return total_score


def evaluate_all_metrics(evaluations, alpha, beta, gamma):

    best_api = None
    best_score = float('-inf')
    results=[]

    print("API Scores:\n")

    for api_name, image_metrics in evaluations.items():
        total_score = 0
        total_score_llm = 0
        total_lev_accuracy = 0
        total_word_accuracy = 0
        total_cer = 0
        total_lev_accuracy_llm = 0
        total_word_accuracy_llm = 0
        total_cer_llm = 0
        num_images = len(image_metrics)
        for image_name, metrics in image_metrics.items():
            total_score += calculate_score(metrics, alpha, beta, gamma)
            total_score_llm += calculate_score_llm(metrics, alpha, beta, gamma)

            total_lev_accuracy += metrics.get("Levenshtein Accuracy", 0)
            total_word_accuracy += metrics.get("Word Accuracy", 0)
            total_cer += metrics.get("Character Error Rate", 0)

            total_lev_accuracy_llm += metrics.get("Levenshtein Accuracy LLM", 0)
            total_word_accuracy_llm += metrics.get("Word Accuracy LLM", 0)
            total_cer_llm += metrics.get("Character Error Rate LLM", 0)
        avg_score = total_score / num_images if num_images > 0 else 0
        avg_score_llm = total_score_llm / num_images if num_images > 0 else 0

        avg_lev_accuracy = total_lev_accuracy / num_images if num_images > 0 else 0
        avg_word_accuracy = total_word_accuracy / num_images if num_images > 0 else 0
        avg_cer = total_cer / num_images if num_images > 0 else 0

        avg_lev_accuracy_llm = total_lev_accuracy_llm / num_images if num_images > 0 else 0
        avg_word_accuracy_llm = total_word_accuracy_llm / num_images if num_images > 0 else 0
        avg_cer_llm = total_cer_llm / num_images if num_images > 0 else 0

        print(f"{api_name} - Average Score: {avg_score:.2f}, with LLM: {avg_score_llm:.2f}")

        results.append({
            "API Name": api_name,
            "Levenshtein Accuracy": avg_lev_accuracy,
            "Levenshtein Accuracy with LLM": avg_lev_accuracy_llm,
            "Word Accuracy": avg_word_accuracy,
            "Word Accuracy with LLM": avg_word_accuracy_llm,
            "Character Error Rate": avg_cer,
            "Character Error Rate with LLM": avg_cer_llm,
            "Average Score": avg_score,
            "Average Score with LLM": avg_score_llm
        })


        if avg_score > best_score:
            best_score = avg_score
            best_api = api_name


    df = pd.DataFrame(results)
    df.to_excel("eng_document_ocr_evaluation_results.xlsx", index=False)

    print("\nResults saved to excel file")
    return best_api, best_score




In [ ]:
# Esempio di dati delle metriche
evaluations = {
    "Azure OCR": Azure_Evaluation,
    "AWS Textract": Aws_Evaluation,
    "OCR.space": OcrSpace_Evaluation,
    "Tesseract": Tesseract_Evaluation,
    "Google Cloud Vision API": OcrCloudVision_Evaluation
}

# Confronta le metriche per tutte le API
best_api, best_score = evaluate_all_metrics(evaluations, alpha=1.3, beta=0.6, gamma=1.5)
print(f"\nThe best OCR API is {best_api} with an average score of {best_score:.2f}.")


API Scores:

Azure OCR - Average Score: 66.96, with LLM: 65.18
AWS Textract - Average Score: 73.62, with LLM: 70.48
OCR.space - Average Score: 27.37, with LLM: 31.22
Tesseract - Average Score: 21.88, with LLM: 18.70
Google Cloud Vision API - Average Score: 41.33, with LLM: 31.51

Results saved to excel file

The best OCR API is AWS Textract with an average score of 73.62.


In [ ]:
avg_google_vision_time=sum(google_vision_time_record)/len(google_vision_time_record)
avg_aws_time=sum(aws_time_record)/len(aws_time_record)
avg_azure_time=sum(azure_time_record)/len(azure_time_record)
avg_tesseract_time=sum(tesseract_time_record)/len(tesseract_time_record)
avg_ocr_space_time=sum(ocr_space_time_record)/len(ocr_space_time_record)

time_data = {
    "Model": ["Azure","AWS" ,"OCR.SPACE" ,"Tesseract" ,"Google Vision"],  # You can add more models here
    "Average Time (seconds)": [avg_azure_time,avg_aws_time,avg_ocr_space_time,avg_tesseract_time,avg_google_vision_time],
}

df = pd.DataFrame(time_data)
df.to_excel("average_ocr_processing_times_docs.xlsx", index=False)

print("Average processing times saved to 'average_processing_times.xlsx'")

Average processing times saved to 'average_processing_times.xlsx'
